# 4.1 Multiplying square matrices

(4.1implementation)=
## Implementing matrix multiplication

Let us implement the two matrix multiplication algorithms ("standard" and recursive) described in the book for good measure. As in the book, we pass matrices by reference to our functions, and consider only square integer matrices of order a power of $2$.

As a point of interest, let us make it so that each algorithm returns the number of multiplications and additions of numbers that are made, just so that we can verify that the cost is $\Theta(n^3)$ (in fact, thenumber of operations for both algorithms is $2n^3$).

Moreover, two techniques for matrix partitioning are mentined in the book:

1. Create submatrices by allocating new memory and copying entries. This is implemented as `matrix_multiply_recursive_copy` below.

    This option uses more resources, as copying and allocating are considered to be constant-time operations. Namely, for a matrix of order $n$, copying it into $4$ submatrices (which corresponds to line 6. of algorithm `MATRIX-MULTIPLY-RECURSIVE` in the book) has cost $\Theta(n^2)$. In any case, the Master Theorem (Theorem 4.1, item 1.) still imples that the overall cost is $\Theta(n^3)$ (see also [Exercise 4.1-3](exercise4.1-3)).
    
    However, copying submatrices also has a memory cost:
    - Each initial $n\times n$ matrices are copied to $4$ $(n/2)\times(n/2)$ submatrices.
    - Each $(n/2)\times(n/2)$ submatrix is copied to $4$ $(n/4)\times(n/4)$ submatrices.
    - Each $(n/4)\times(n/4)$ submatrix is copied to $4$ $(n/8)\times(n/8)$ submatrices.
    
    and so on, until we get to a base case. This adds up - for each initial matrix of order $n\times n$ - to a memory cost (throughout almost the whole algorithm) of
    \begin{equation*}\sum_{i=1}^{\lg n} 4 \cdot\left(\dfrac{n}{2^i}\right)^2 = \dfrac{4}{3}(n^2-1)\end{equation*}
    /(assuming $n$ a power of $2$), which essentially multiplies memory usage by $7/3$, but still is quite controlled.
    
2. Use index calculations for submatrices, which is implemented below as `matrix_multiply_recursive_index`

    This is to say that, instead of simply giving $A=\begin{bmatrix}a_{ij}\end{bmatrix}_{ij}$ as an argument, we use a matrix $A$ as well as indexes $i_1,i_2,j_1$ and $j_2$ as arguments, to mean that the matrix being multiplied is in fact the submatrix consisting of the entries of $A$ between rows $i_1$ and $i_2$ and columns $j_1$ and $j_2$.
    
    As a downside, this approach makes the code messier.

In [1]:
#include <stdio.h>
#include <time.h>

int matrix_multiply ( int ** A , int ** B , int ** C , size_t n ) {
    // Standard matrix multiplication
    
    int ops = 0; // number of operations
    size_t i,j,k; // indices
    for (i=0;i<n;i++) {
        for (j=0;j<n;j++) {
            for (k=0;k<n;k++) {
                C[i][j] += A[i][k]*B[k][j];
                ops+=2;
            }
        }
    }
    return ops;
}

// Function that alocates a copy of a submatrix dynamically

int ** submatrix ( int ** A , size_t i1 , size_t i2 , size_t j1 , size_t j2 ) {
    // Creates a submatrix A[i1:i2,j1:j2] (inclusive)
    size_t i,j; // indices
    size_t m = i2-i1+1;
    size_t n = j2-j1+1;
    
    int ** A_ = malloc(m * sizeof(int *));
    
    for (i=0;i<m;i++) {
        A_[i] = malloc(n*sizeof(int));
        for (j=0;j<n;j++) {
            A_[i][j]=A[i+i1][j+j1];
        }
    }
            
    return A_;
}

// Free dinamically allocated matrix for convenience

void free_allocated_matrix(int ** A , size_t m) {
    // Frees a dinamically allocated matrix A with m rows.
    size_t i;
    
    for (i=0;i<m;i++) {
        free(A[i]);
    }
    
    free(A);
}

int matrix_multiply_recursive_copy ( int ** A , int ** B , int ** C , size_t n ) {
    
    //Works only for powers of 2
    
    
    if (n==1) {
        C[0][0] += A[0][0]*B[0][0];
        return 2;
    }
    
    size_t i,j; // Indices
    
    int ops = 0; // number of operations
    
    //Create submatrices
    size_t m = n/2;
    
    int ** A11 = submatrix(A,0,m-1,0,m-1);
    int ** A12 = submatrix(A,0,m-1,m,n-1);
    int ** A21 = submatrix(A,m,n-1,0,m-1);
    int ** A22 = submatrix(A,m,n-1,m,n-1);
    int ** B11 = submatrix(B,0,m-1,0,m-1);
    int ** B12 = submatrix(B,0,m-1,m,n-1);
    int ** B21 = submatrix(B,m,n-1,0,m-1);
    int ** B22 = submatrix(B,m,n-1,m,n-1);
    int ** C11 = submatrix(C,0,m-1,0,m-1);
    int ** C12 = submatrix(C,0,m-1,m,n-1);
    int ** C21 = submatrix(C,m,n-1,0,m-1);
    int ** C22 = submatrix(C,m,n-1,m,n-1);
    
    ops += matrix_multiply_recursive_copy(A11,B11,C11,m);
    ops += matrix_multiply_recursive_copy(A12,B21,C11,m);
    ops += matrix_multiply_recursive_copy(A11,B12,C12,m);
    ops += matrix_multiply_recursive_copy(A12,B22,C12,m);
    ops += matrix_multiply_recursive_copy(A21,B11,C21,m);
    ops += matrix_multiply_recursive_copy(A22,B21,C21,m);
    ops += matrix_multiply_recursive_copy(A21,B12,C22,m);
    ops += matrix_multiply_recursive_copy(A22,B22,C22,m);
    
    // Copy the results back to the original matrices
    
    for (i=0;i<m;i++) {
        for (j=0;j<m;j++) {
            A[i][j] = A11[i][j];
            B[i][j] = B11[i][j];
            C[i][j] = C11[i][j];
        }
        for (;j<n;j++) {
            A[i][j] = A12[i][j-m];
            B[i][j] = B12[i][j-m];
            C[i][j] = C12[i][j-m];
        }
    }
    for (;i<n;i++) {
        for (j=0;j<m;j++) {
            A[i][j] = A21[i-m][j];
            B[i][j] = B21[i-m][j];
            C[i][j] = C21[i-m][j];
        }
        for (;j<n;j++) {
            A[i][j] = A22[i-m][j-m];
            B[i][j] = B22[i-m][j-m];
            C[i][j] = C22[i-m][j-m];
        }
    }    
    
    free_allocated_matrix(A11,m);
    free_allocated_matrix(A12,m);
    free_allocated_matrix(A21,n-m);
    free_allocated_matrix(A22,n-m);
    free_allocated_matrix(B11,m);
    free_allocated_matrix(B12,m);
    free_allocated_matrix(B21,n-m);
    free_allocated_matrix(B22,n-m);
    free_allocated_matrix(C11,m);
    free_allocated_matrix(C12,m);
    free_allocated_matrix(C21,n-m);
    free_allocated_matrix(C22,n-m);
    
    return ops;
}

int matrix_multiply_recursive_index ( int ** A , size_t iA1 , size_t iA2 , size_t jA1 , size_t jA2 ,
                                      int ** B , size_t iB1 , size_t iB2 , size_t jB1 , size_t jB2 ,
                                      int ** C , size_t iC1 , size_t iC2 , size_t jC1 , size_t jC2 ) {
    
    // Works only for powers of 2
    // Assumes submatrices are of same order
    /*
        Takes submatrices of A and B and adds their product to a submatrix of C
        Indexes i*1 ad i*2 determine that the submatrix has rows of index i
        satisfying
            i1 <= i < i2
        and similarly for j*1 and j*2 
        
        To take all of A (square of order n by n), simply use iA1 = jA1 = 0 and iA2 = jA2 = n
    */
    
    size_t n = iA2-iA1; // order of all matrices
    
    if (n==1) {
        C[iC1][jC1] += A[iA1][jA1]*B[iB1][jB1];
        return 2;
    }
    
    int ops = 0; // number of operations
    
    //Create submatrices
    size_t m = n/2; // half of order
    
    /*
        A11 = A[i1:i1+m][j1:j1+m]
        A12 = A[i1:i1+m][j1+m:j2]
        A11 = A[i1+m:i2][j1:j1+m]
        A12 = A[i1+m:i2][j1+m:j2]
        
        and similarly for B11, B12 etc.
    */
    
    //C11 += A11*B11
    ops += matrix_multiply_recursive_index( A , iA1 , iA1+m , jA1 , jA1+m ,
                                            B , iB1 , iB1+m , jB1 , jB1+m ,
                                            C , iC1 , iC1+m , jC1 , jC1+m );
    //C11 += A12*B21
    ops += matrix_multiply_recursive_index( A , iA1 , iA1+m , jA1+m , jA2 ,
                                            B , iB1+m , iB2 , jB1 , jB1+m ,
                                            C , iC1 , iC1+m , jC1 , jC1+m);
    //C12 += A11*B12
    ops += matrix_multiply_recursive_index( A , iA1 , iA1+m , jA1 , jA1+m ,
                                            B , iB1 , iB1+m , jB1+m , jB2 ,
                                            C , iC1 , iC1+m , jC1+m , jC2 );
    //C12 += A12*B22
    ops += matrix_multiply_recursive_index( A , iA1 , iA1+m , jA1+m , jA2 ,
                                            B , iB1+m , jB2 , jB1+m , jB2 ,
                                            C , iC1 , iC1+m , jC1+m , jC2 );
    //C21 += A21*B11
    ops += matrix_multiply_recursive_index( A , iA1+m , iA2 , jA1 , jA1+m ,
                                            B , iB1 , iB1+m , jB1 , jB1+m ,
                                            C , iC1+m , iC2 , jC1 , jC1+m );
    //C21 += A22*B21
    ops += matrix_multiply_recursive_index( A , iA1+m , iA2 , jA1+m , jA2 ,
                                            B , iB1+m , iB2 , jB1 , jB1+m ,
                                            C , iC1+m , iC2 , jC1 , jC1+m );
    //C22 += A21*B12
    ops += matrix_multiply_recursive_index( A , iA1+m , iA2 , jA1 , jA1+m ,
                                            B , iB1 , iB1+m , jB1+m , jB2 ,
                                            C , iC1+m , iC2 , jC1+m , jC2 );
    //C22 += A22*B22
    ops += matrix_multiply_recursive_index( A , iA1+m , iA2 , jA1+m , jA2 ,
                                            B , iB1+m , iB2 , jB1+m , jB2 ,
                                            C , iC1+m , iC2 , jC1+m , jC2 );
    
    
    return ops;
}

// print matrix nicely

void print_matrix (int ** A , size_t m , size_t n ) {
    //prints a matrix A in a nice, python-compatible format
    size_t i,j; // indices
    printf("[\n");
    for (i=0;i<m-1;i++) {
        printf("  [");
        for (j=0;j<n-1;j++) {
            printf(" %6d ," , A[i][j]);
        }
        printf(" %6d],\n" , A[i][j]);
    }
    
    // last row
    printf("  [");
    for (j=0;j<n-1;j++) {
        printf(" %6d ," , A[i][j]);
    }
    printf(" %6d]\n" , A[i][j]);
    
    printf("]");
    
}

int main() {
    srand(time(NULL)); // random seed
    
    // Order of matrices; 8 is large-ish and readable
    size_t n = 8;
    
    size_t i,j; // indices
    
    // Create random matrices with entries between -99 and 99
    int ** A , ** B , ** C , ** D , **E;
    
    A = malloc(n*sizeof(int*));
    B = malloc(n*sizeof(int*));
    C = malloc(n*sizeof(int*));
    D = malloc(n*sizeof(int*));
    E = malloc(n*sizeof(int*));
    
    for (i=0;i<n;i++) {
        A[i] = malloc(n*sizeof(int));
        B[i] = malloc(n*sizeof(int));
        C[i] = malloc(n*sizeof(int));
        D[i] = malloc(n*sizeof(int));
        E[i] = malloc(n*sizeof(int));
        
        for (j=0;j<n;j++) {
            // between -99 and 99
            A[i][j] = rand()%199 - 99;
            B[i][j] = rand()%199 - 99;
            C[i][j] = 0;
            D[i][j] = 0;
            E[i][j] = 0;
        }
    }
    
    // Start calculating products and number of operations
    int number_of_operations;
    
    for (i=0;i<30;i++) {
        printf("=");
    }
    printf("\n");
    
    printf("Random matrices:\n");
    printf("A =\n");
    print_matrix(A,n,n);
    
    printf("\n");
    
    printf("B =\n");
    print_matrix(B,n,n);
    
    // ==============================
    printf("\n");
    for (i=0;i<30;i++) {
        printf("=");
    }
    printf("\n");

    printf("Matrix product calculated through standard matrix multiplication:\n");
    
    number_of_operations = matrix_multiply(A,B,C,n);
    
    printf("A*B =\n");
    print_matrix(C,n,n);
    
    printf("\n");
    
    printf("Number of operations: %d" , number_of_operations);
    
    // ==============================
    printf("\n");
    for (i=0;i<30;i++) {
        printf("=");
    }
    printf("\n");
    printf("Matrix product calculated through recursive matrix multiplication with copying submatrices:\n");
    
    number_of_operations = matrix_multiply_recursive_copy(A,B,D,n);
    
    printf("A*B =\n");
    print_matrix(D,n,n);
    
    printf("\n");
    
    printf("Number of operations: %d" , number_of_operations);
    
    // ==============================
    printf("\n");
    for (i=0;i<30;i++) {
        printf("=");
    }
    printf("\n");
    printf("Matrix product calculated through recursive matrix multiplication with index calculations:\n");
    
    number_of_operations = matrix_multiply_recursive_index( A , 0 , n , 0 , n ,
                                                            B , 0 , n , 0 , n ,
                                                            E , 0 , n , 0 , n );
    
    printf("A*B =\n");
    print_matrix(E,n,n);
    
    printf("\n");
    
    printf("Number of operations: %d" , number_of_operations);
    
    // ==============================
    printf("\n");
    for (i=0;i<30;i++) {
        printf("=");
    }
    printf("\n");
    
    for (i=0;i<n;i++) {
        for (j=0;j<n;j++) {
            if (C[i][j]!=D[i][j] || C[i][j]!=E[i][j]) {
                printf("Entries (i,j)=(%zu,%zu) were calculated differently!\n",i,j);
                i=n+1;
                j=n+1;
            }
        }
    }
    if (i==n && j==n) {
        printf("The products were calculated the same.");
    }
    printf("\n");
    for (i=0;i<n;i++) {
        free(A[i]);
        free(B[i]);
        free(C[i]);
        free(D[i]);
        free(E[i]);
    }
    
    
    
    free(A);
    free(B);
    free(C);
    free(D);
    free(E);
    
    return 0;
}

Random matrices:
A =
[
  [     37 ,    -59 ,     45 ,     67 ,      3 ,     -1 ,    -40 ,    -45],
  [     77 ,     59 ,     74 ,    -35 ,     91 ,    -93 ,    -90 ,     18],
  [    -37 ,     41 ,    -47 ,     55 ,     67 ,     35 ,     92 ,     71],
  [    -63 ,     83 ,     34 ,    -71 ,     93 ,     12 ,    -29 ,     26],
  [     65 ,     -6 ,     52 ,     28 ,    -21 ,     -1 ,    -70 ,    -83],
  [     78 ,    -46 ,    -97 ,     91 ,    -52 ,     51 ,    -13 ,    -95],
  [     -2 ,    -75 ,     23 ,     22 ,     68 ,    -73 ,    -55 ,    -58],
  [     97 ,     12 ,      8 ,     48 ,     59 ,    -26 ,     30 ,    -24]
]
B =
[
  [    -96 ,    -81 ,     -5 ,     71 ,    -18 ,    -74 ,     61 ,     91],
  [    -30 ,    -85 ,    -96 ,    -81 ,     68 ,    -89 ,    -41 ,     46],
  [     58 ,     84 ,      8 ,     33 ,    -46 ,    -96 ,     90 ,     69],
  [     31 ,    -89 ,    -74 ,      2 ,    -88 ,     79 ,    -92 ,     10],
  [    -56 ,     95 ,    -51 ,     -4 ,    -37 ,    -29 , 

Matrix product calculated through recursive matrix multiplication with index calculations:
A*B =
[
  [   4000 ,  -1427 ,    508 ,   3261 , -16570 ,   1846 ,   6741 ,  10696],
  [ -18536 ,  13960 ,  -6758 , -10102 ,  -7953 , -34905 ,  18045 ,  21479],
  [  -1928 ,  -2233 , -11766 ,   6107 ,   6450 ,  11886 , -25057 , -20226],
  [  -1625 ,  20893 ,  -8711 , -10559 ,   4942 , -18299 ,   -423 ,  -3551],
  [   1387 ,  -7993 ,   -664 ,  -3293 , -12344 ,  -8926 ,  14542 ,  21613],
  [    612 , -31439 ,  -1349 ,   3778 ,  -8340 ,  21010 ,  -6388 ,   6359],
  [  -3237 ,  10368 ,   4783 ,  -6385 , -14391 ,    751 ,  10017 ,   7773],
  [ -12414 , -11750 ,  -5744 ,   5193 ,  -4251 ,  -3189 ,    590 ,   9104]
]
Number of operations: 1024
The products were calculated the same.


## 4.1-1

> Generalize `MATRIX-MULTIPLY-RECURSIVE` to multiply $n\times n$ matrices for which $n$ is not necessarily an exact power of $2$. Give a recurrence describing its running time. Argue that it runs in $\Theta(n^3)$ time in the worst case.

For the general case, we will need to be able to multiply rectangular matrices. This is so because we need to be able to break up a matrix into smaller ones and operate on block matrices recursively until we get to $1\times 1$ matrices.

For example, if we start with a $3\times 3$ matrix and try to break it up into four submatrices, we will end up with a $2\times 1$ and a $1\times 2$ matrix, which we should be able to operate with.

So we will deal with three arguments for the order of the matrices: $m$ , $n$ and $p$. We assume that $A$ is $m\times n$, $B$ is $n\times p$ and $C=AB$ is $m\times p$. In this case, it is also useful to allow for "empty matrices", since the algorithm might end up trying to divide a single column or row into two submatrices, one of which will be empty. (An alternative would be to consider cases of rows and/or columns separately inside an "if-else" thread.)

    MATRIX-MULTIPLY-RECURSIVE ( A , B , C , m , n , p)
    1.  if (m==0) or (n==0) or (p==0):
    2.      return
    3.  if (m==1) and (n==1) and (p==1):
    4.      c[1][1] += a[1][1]*b[1][1]
    5.      return
    6.  Let m_1 = floor(m/2),
            m_2 = m - m_1,
            n_1 = floor(n/2),
            n_2 = n - n_1,
            p_1 = floor(p/2),
            p_2 = p - p_1,
    7.  Partition A, B and C into submatrices
            A_11, A_12, A_21, A_22,
            B_11, B_12, B_21, B_22,
            C_11, C_12, C_21, C_22
        with A_ij of order m_i x n_j
             B_ij of order n_i x p_j
             C_ij of order m_i x p_j
    8.  for i = 1, 2
    9.      for j = 1, 2
    10.         for k = 1, 2
    11.             MATRIX-MULTIPLY-RECURSIVE ( A_ik , B_kj , C_ij , m_i , n_k , p_j )

Let us implement this algorithm. We can adapt the last algorithm [above](4.1implementation), so that the orders of the (sub)matrices involved need not be explicit, as they can be immediately calculated from the index arguments.

In [2]:
#include <stdio.h>
#include <time.h>

int matrix_multiply ( int ** A , int ** B , int ** C , size_t m , size_t n , size_t p ) {
    /*
        Standard matrix multiplication
        A of order m x n , B of order n x p , C += A*B of order m x p
    */
    
    int ops = 0; // number of operations
    size_t i,j,k; // indices
    for (i=0;i<m;i++) {
        for (j=0;j<p;j++) {
            for (k=0;k<n;k++) {
                C[i][j] += A[i][k]*B[k][j];
                ops+=2;
            }
        }
    }
    return ops;
}

int matrix_multiply_recursive ( int ** A , size_t iA1 , size_t iA2 , size_t jA1 , size_t jA2 ,
                                int ** B , size_t iB1 , size_t iB2 , size_t jB1 , size_t jB2 ,
                                int ** C , size_t iC1 , size_t iC2 , size_t jC1 , size_t jC2 ) {
    
    // Works only for powers of 2
    // Assumes submatrices are of same order
    /*
        Takes submatrices of A and B and adds their product to a submatrix of C
        Indexes i*1 ad i*2 determine that the submatrix has rows of index i
        satisfying
            i1 <= i < i2
        and similarly for j*1 and j*2 
        
        To take all of A (square of order n by n), simply use iA1 = jA1 = 0 and iA2 = jA2 = n
    */
    
    // Matrix orders
    size_t m = iA2-iA1;
    size_t n = iB2-iB1;
    size_t p = jB2-jB1;
    
    if ( m==0 || n==0 || p==0 ) {
        return 0;
    }
    
    if ( m==1 && n==1 && p==1 ) {
        C[iC1][jC1] += A[iA1][jA1]*B[iB1][jB1];
        return 2;
    }
    
    int ops = 0; // number of operations
    
    //Create submatrices
    size_t m_ = m/2; // half of order
    size_t n_ = n/2; // half of order
    size_t p_ = p/2; // half of order
    
    /*
        A11 = A[i1:i1+m][j1:j1+m]
        A12 = A[i1:i1+m][j1+m:j2]
        A11 = A[i1+m:i2][j1:j1+m]
        A12 = A[i1+m:i2][j1+m:j2]
        
        and similarly for B11, B12 etc.
    */
    
    //C11 += A11*B11
    ops += matrix_multiply_recursive( A , iA1 , iA1+m_ , jA1 , jA1+n_ ,
                                            B , iB1 , iB1+n_ , jB1 , jB1+p_ ,
                                            C , iC1 , iC1+m_ , jC1 , jC1+p_ );
    //C11 += A12*B21
    ops += matrix_multiply_recursive( A , iA1 , iA1+m_ , jA1+n_ , jA2 ,
                                            B , iB1+n_ , iB2 , jB1 , jB1+p_ ,
                                            C , iC1 , iC1+m_ , jC1 , jC1+p_);
    //C12 += A11*B12
    ops += matrix_multiply_recursive( A , iA1 , iA1+m_ , jA1 , jA1+n_ ,
                                            B , iB1 , iB1+n_ , jB1+p_ , jB2 ,
                                            C , iC1 , iC1+m_ , jC1+p_ , jC2 );
    //C12 += A12*B22
    ops += matrix_multiply_recursive( A , iA1 , iA1+m_ , jA1+n_ , jA2 ,
                                            B , iB1+n_ , iB2 , jB1+p_ , jB2 ,
                                            C , iC1 , iC1+m_ , jC1+p_ , jC2 );
    //C21 += A21*B11
    ops += matrix_multiply_recursive( A , iA1+m_ , iA2 , jA1 , jA1+n_ ,
                                            B , iB1 , iB1+n_ , jB1 , jB1+p_ ,
                                            C , iC1+m_ , iC2 , jC1 , jC1+p_ );
    //C21 += A22*B21
    ops += matrix_multiply_recursive( A , iA1+m_ , iA2 , jA1+n_ , jA2 ,
                                            B , iB1+n_ , iB2 , jB1 , jB1+p_ ,
                                            C , iC1+m_ , iC2 , jC1 , jC1+p_ );
    //C22 += A21*B12
    ops += matrix_multiply_recursive( A , iA1+m_ , iA2 , jA1 , jA1+n_ ,
                                            B , iB1 , iB1+n_ , jB1+p_ , jB2 ,
                                            C , iC1+m_ , iC2 , jC1+p_ , jC2 );
    //C22 += A22*B22
    ops += matrix_multiply_recursive( A , iA1+m_ , iA2 , jA1+n_ , jA2 ,
                                            B , iB1+n_ , iB2 , jB1+p_ , jB2 ,
                                            C , iC1+m_ , iC2 , jC1+p_ , jC2 );
    
    return ops;
}

// print matrix nicely

void print_matrix (int ** A , size_t m , size_t n ) {
    //prints a matrix A in a nice, python-compatible format
    size_t i,j; // indices
    printf("[\n");
    for (i=0;i<m-1;i++) {
        printf("  [");
        for (j=0;j<n-1;j++) {
            printf(" %6d ," , A[i][j]);
        }
        printf(" %6d],\n" , A[i][j]);
    }
    
    // last row
    printf("  [");
    for (j=0;j<n-1;j++) {
        printf(" %6d ," , A[i][j]);
    }
    printf(" %6d]\n" , A[i][j]);
    
    printf("]");
    
}

/////////////////

int main() {
    srand(time(NULL)); // random seed
    
    // Order of matrices;
    
    size_t m = 4 + rand()%15;
    size_t n = 4 + rand()%15;
    size_t p = 4 + rand()%15;

    size_t i,j; // indices

    // Create random matrices with entries between -99 and 99
    int ** A , ** B , ** C , **E;

    A = malloc(m*sizeof(int*));
    B = malloc(n*sizeof(int*));
    C = malloc(m*sizeof(int*));
    E = malloc(m*sizeof(int*));

    for (i=0;i<m;i++) {
        A[i] = malloc(n*sizeof(int));
        for (j=0;j<n;j++) {
            A[i][j] = rand()%199 - 99;
        }
    }

    for (i=0;i<n;i++) {
        B[i] = malloc(p*sizeof(int));
        for (j=0;j<p;j++) {
            B[i][j] = rand()%199 - 99;
        }
    }

    for (i=0;i<m;i++) {
        C[i] = malloc(p*sizeof(int));
        E[i] = malloc(p*sizeof(int));
        for (j=0;j<p;j++) {
            C[i][j] = 0;
            E[i][j] = 0;
        }
    }

    // Start calculating products and number of operations
    int number_of_operations;

    for (i=0;i<30;i++) {
        printf("=");
    }

    printf("\n");

    printf("Random matrices:\n");
    printf("A =\n");
    print_matrix(A,m,n);

    printf("\n");

    printf("B =\n");
    print_matrix(B,n,p);

    // ==============================
    printf("\n");
    for (i=0;i<30;i++) {
        printf("=");
    }
    printf("\n");

    printf("Matrix product calculated through standard matrix multiplication:\n");

    number_of_operations = matrix_multiply(A,B,C,m,n,p);

    printf("A*B =\n");
    print_matrix(C,m,p);

    printf("\n");

    printf("Number of operations: %d" , number_of_operations);

    // ==============================
    printf("\n");
    for (i=0;i<30;i++) {
        printf("=");
    }
    printf("\n");
    printf("Matrix product calculated through recursive matrix multiplication with index calculations:\n");

    number_of_operations = matrix_multiply_recursive( A , 0 , m , 0 , n ,
                                                      B , 0 , n , 0 , p ,
                                                      E , 0 , m , 0 , p );

    printf("A*B =\n");
    print_matrix(E,m,p);

    printf("\n");

    printf("Number of operations: %d" , number_of_operations);

    // ==============================
    printf("\n");
    for (i=0;i<30;i++) {
        printf("=");
    }
    printf("\n");

    for (i=0;i<m;i++) {
        for (j=0;j<p;j++) {
            if (C[i][j]!=E[i][j]) {
                printf("Entries (i,j)=(%zu,%zu) were calculated differently!\n",i,j);
                i=n+1;
                j=p+1;
            }
        }
    }
    if (i==m && j==p) {
        printf("The products were calculated the same.");
    }
    printf("\n");

    for (i=0;i<m;i++) {
        free(A[i]);
        free(C[i]);
        free(E[i]);
    }

    for (i=0;i<n;i++) {
        free(B[i]);
    }

    free(A);
    free(B);
    free(C);
    free(E);

    return 0;
}

Random matrices:
A =
[
  [    -81 ,     45 ,     -5 ,     67 ,     71 ,      3 ,    -18 ,     -1 ,    -74 ,    -40 ,     61 ,    -45 ,     91 ,     77],
  [    -30 ,     59 ,    -85 ,     74 ,    -96 ,    -35 ,    -81 ,     91 ,     68 ,    -93 ,    -89 ,    -90 ,    -41 ,     18],
  [     46 ,    -37 ,     58 ,     41 ,     84 ,    -47 ,      8 ,     55 ,     33 ,     67 ,    -46 ,     35 ,    -96 ,     92],
  [     90 ,     71 ,     69 ,    -63 ,     31 ,     83 ,    -89 ,     34 ,    -74 ,    -71 ,      2 ,     93 ,    -88 ,     12],
  [     79 ,    -29 ,    -92 ,     26 ,     10 ,     65 ,    -56 ,     -6 ,     95 ,     52 ,    -51 ,     28 ,     -4 ,    -21],
  [    -37 ,     -1 ,    -29 ,    -70 ,    -29 ,    -83 ,    -33 ,     78 ,     77 ,    -46 ,     12 ,    -97 ,    -40 ,     91],
  [     72 ,    -52 ,    -20 ,     51 ,     18 ,    -13 ,    -46 ,    -95 ,    -47 ,     -2 ,     -2 ,    -75 ,    -73 ,     23],
  [     29 ,     22 ,     79 ,     68 ,     97 ,    -73 ,     98 , 

  [     23 ,     38 ,     99 ,    -21 ,     -2 ,     -1 ,     30 ,     21 ,    -71 ,     85 ,     73 ,    -67 ,     15 ,    -50 ,    -37 ,     45],
  [    -68 ,     91 ,     54 ,    -95 ,     -6 ,     90 ,     30 ,    -58 ,    -31 ,     -4 ,    -87 ,     22 ,     11 ,    -52 ,     73 ,    -65],
  [     86 ,     73 ,     13 ,    -39 ,    -51 ,    -57 ,     81 ,    -46 ,    -95 ,     31 ,    -37 ,     -3 ,     57 ,     25 ,    -58 ,     88],
  [     -7 ,     96 ,     69 ,     86 ,     86 ,     -1 ,    -94 ,    -68 ,     95 ,    -82 ,     53 ,    -17 ,    -35 ,     27 ,     -6 ,    -72],
  [    -23 ,     84 ,    -12 ,     25 ,    -96 ,    -54 ,     55 ,    -91 ,     53 ,    -82 ,      5 ,     10 ,     19 ,     46 ,     -2 ,    -88],
  [     19 ,    -32 ,     74 ,    -18 ,     43 ,     79 ,     14 ,     15 ,     73 ,    -56 ,     98 ,    -84 ,     70 ,     -8 ,    -57 ,    -53],
  [    -24 ,      8 ,     48 ,    -43 ,     30 ,      3 ,    -35 ,    -16 ,     -3 ,     46 ,     93 ,    -84 , 

## 4.1-2

> How quickly can you multiply a $kn\times n$ matrix ( $kn$ rows and $n$ columns) by an $n\times kn$ matrix, where $k\geq 1$, using `MATRIX-MULTIPLY-RECURSIVE` as a subroutine? Answer the same question for multiplying an $n\times kn$ matrix by a $kn\times n$ matrix. Which is asymptotically faster, and by how much?

Let $A$ be a $kn\times n$ matrix and $B$ be a $n\times kn$ matrix. Then, we can partition $A$ and $B$ into $n\times n$ matrices as
\begin{equation*}A=\begin{bmatrix}A_1\\A_2\\\vdots\\A_k\end{bmatrix}\qquad\text{and}\qquad B=\begin{bmatrix}B_1&B_2&\cdots&B_k\end{bmatrix}.\end{equation*}

Then $AB$ is the $kn\times kn$ block matrix
\begin{equation*}AB=\begin{bmatrix}A_1B_1&A_1B_2&\cdots&A_1B_k\\A_2B_1&A_2B_2&\cdots&A_2B_k\\\vdots&\vdots&\ddots&\vdots\\A_kB_1&A_kB_2&\cdots&A_kB_k\end{bmatrix},\end{equation*}
and each product $A_iB_j$ may be computed with `MATRIX-MULTIPLY-RECURSIVE` in $\Theta(n^3)$ time. Since there are $k^2$ such products, this is (approximately) $k^2$ times slower than a single application of `MATRIX-MULTIPLY-RECURSIVE` takes.

Similarly, $BA$ is the $n\times n$ matrix
\begin{equation*}BA=B_1A_1+B_2A_2+\cdots+B_kA_k.\end{equation*}
Each term $B_iA_i$ may b computed with `MATRIX-MULTIPLY-RECURSIVE`. There are $k$ such terms to compute, and adding them up takes time $\Theta(kn^2)$, which is asymptotically negligible. Thus, computing $BA$ takes (approximately, asymptotically) $k$ times slower than a single application of `MATRIX-MULTIPLY-RECURSIVE` takes.

In this manner, multiplying a $kn\times n$ matrix by a $n\times kn$ matrix is (asymptotically) $k$ times slower than multiplying a $n\times kn$ matrix by a $kn\times n$ matrix, even though the running times for both problems are $\Theta(n^3)$.

## 4.1-3

> Suppose that instead of partitioning matrices by index calculation in `MATRIX-MULTIPLY-RECURSIVE`, you copy the appropriate elements of $A$, $B$, and $C$ into separate $n/2\times n/2$ submatrices $A_{11},A_{12},A_{21},A_{22};B_{11},B_{12},B_{21},B_{22};C_{11},C_{12},C_{21},C_{22}$, respectively. After the recursive calls, you copy the results from $C_{11},C_{12},C_{21}$, and $C_{22}$ back into the appropriate places. How does recurrence (4.9) change, and what is its solution?

This version of the algorithm was already implemented [above](4.1implementation). The pseudocode is

    MATRIX-MULTIPLY-RECURSIVE(A,B,C,n)
    1   if n == 1
    2   // Base case.
    3       C[1,1] = C[1,1] + A[1,1]*B[1,1]
    4       return
    5   // Divide.
    6   copy A, B, and C into n/2 x n/2 submatrices
        A11, A12, A21, A22; B11, B12, B21, B22;
        and C11, C12, C21, C22; respectively
    7   // Conquer.
    8   MATRIX-MULTIPLY-RECURSIVE(A11, B11, C11, n/2)
    9   MATRIX-MULTIPLY-RECURSIVE(A11, B12, C12, n/2)
    10  MATRIX-MULTIPLY-RECURSIVE(A21, B11, C21, n/2)
    11  MATRIX-MULTIPLY-RECURSIVE(A21, B12, C22, n/2)
    12  MATRIX-MULTIPLY-RECURSIVE(A12, B21, C11, n/2)
    13  MATRIX-MULTIPLY-RECURSIVE(A12, B22, C12, n/2)
    14  MATRIX-MULTIPLY-RECURSIVE(A22, B21, C21, n/2)
    15  MATRIX-MULTIPLY-RECURSIVE(A22, B22, C22, n/2)
    16  copy C11, C12, C21, C22 back into C.
    
The only difference is in lines 6 and 16. Both take time $\Theta(n^2)$ instead of time $\Theta(1)$. Thus, the recurrence for the running time of this version of the algorithm is
\begin{equation*}T(n) = 8T(n/2) +\Theta(n^2).\end{equation*}
By the Master Theorem (Theorem 4.1, item 1.), the solution to this recurrence is still $T(n)=\Theta(n^3)$.

## 4.1-4

> Write pseudocode for a divide-and-conquer algorithm `MATRIX-ADD-RECURSIVE` that sums two $n\times n$ matrices $A$ and $B$ by partitioning each of them into four $n/2\times n/2$ submatrices and then recursively summing corresponding pairs of sub-matrices. Assume that matrix partitioning uses $\Theta(1)$-time index calculations. Write a recurrence for the worst-case running time of `MATRIX-ADD-RECURSIVE` and solve your recurrence. What happens if you use $\Theta(n^2)$-time copying to implement the partitioning instead of index calculations?

    MATRIX-ADD-RECURSIVE(A,B,C,n)
    1   // Base case.
    2   if n==1
    3       C[1,1] = C[1,1] + A[1,1] + B[1,1]
    4       return
    5   // Divide.
    6   partition A, B, and C into n/2 x n/2 submatrices
            A11, A12, A21, A22; B11, B12, B21, B22;
            and C11, C12, C21, C22; respectively
    7   // Conquer.
    8   MATRIX-ADD-RECURSIVE(A11, B11, C11, n/2)
    9   MATRIX-ADD-RECURSIVE(A12, B12, C12, n/2)
    10  MATRIX-ADD-RECURSIVE(A21, B21, C21, n/2)
    11  MATRIX-ADD-RECURSIVE(A22, B22, C22, n/2)
    
The recursion for the running-time of this algorithm is
(equation4.1-4(1))=
\begin{equation*}T(n) = 4T(n/2) + \Theta(1).\tag{4.1-4(1)}\end{equation*}
By the Master Theorem, the solution to this recurrence is $T(n)=\Theta(n^2)$.

If we use $\Theta(n^2)$-time copying of submatrices instead of index calculations, the running-time recursion becomes
(equation4.1-4(2))=
\begin{equation*}T(n) = 4T(n/2)+\Theta(n^2).\tag{4.1-4(2)}\end{equation*}
By the Master Theorem, the solution to this recurrence is $T(n)=\Theta(n^2\lg n)$.

```{note}
We may solve recusions [4.1-4(1)](equation4.1-4(1)) and [4.1-4(2)](equation4.1-4(2)) in a manner similar to that of Chapter 2. The first recursion becomes essentially
\begin{equation*}T(n)=4T(n/2)+c.\end{equation*}
Thus,
\begin{align*}
    T(n)
        &= 4T(n/2) + c\\
        &=4(4T(n/2^2)+c)+c\\
        &=4(4(4T(n/2^3)+c)+c)+c
\end{align*}
and so on. So, assuming that $n$ is a power of $2$,
\begin{align*}
    T(n)
        &= 4^{\lg n} T(1) + \left(\sum_{i=0}^{\lg n - 1}4^i\right)c\\
        &= 4^{\lg n} T(1) + \dfrac{4^{\lg n}-1}{4-1}c\\
        &= n^2 T(1) + \dfrac{n^2 - 1}{3}c,
\end{align*}
which is $\Theta(n^2)$.

The second recursion becomes essentially
\begin{equation*}T(n)=4T(n/2)+cn^2.\end{equation*}
Thus,
\begin{align*}
    T(n)
        &= 4T(n/2) + cn^2\\
        &=4(4T(n/2^2)+c(n/2)^2)+cn^2\\
        &=4(4(4T(n/2^3)+c(n/2^2)^2)+c(n/2)^2)+cn^2
\end{align*}
and so on. So, assuming that $n$ is a power of $2$,
\begin{align*}
    T(n)
        &= 4^{\lg n} T(1) + \left(\sum_{i=0}^{\lg n - 1}4^i\left(\dfrac{n}{2^i}\right)^2\right)c\\
        &= 4^{\lg n} T(1) + \left(\sum_{i=0}^{\lg n - 1}n^2\right)c\\
        &= n^2 T(1) + (n^2\lg n) c
\end{align*}
which is $\Theta(n^2\lg n)$.
```